In [1]:
# gensim modules
from gensim import utils
from gensim.models.doc2vec import LabeledSentence
from gensim.models import Doc2Vec
import gensim
import nltk
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize 
from konlpy.utils import pprint
import time
from konlpy.tag import Kkma, Twitter, Komoran
import csv

# numpy
import numpy

#정규식
import re

# random
from random import shuffle

C:\Users\rjsfu\Anaconda3\lib\site-packages\gensim\utils.py:1209: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [6]:
#label과 data 읽기
from os import listdir
from os.path import isfile, join

#문서의 label 저장하기
year = [2008,2014,2017]
docLabels = []
for i in year:
    my_path = 'C:/Users/rjsfu/Dropbox/과제/텍스트마이닝/data/신문기사/매일경제/(웅진코웨이)%C4%DA%BF%FE%C0%CC/' + str(i)
    for  i in listdir(my_path):
        if i.endswith('.txt'):
            docLabels.append(i)

#data 저장하기
data  = []
for i in year:
    my_path = 'C:/Users/rjsfu/Dropbox/과제/텍스트마이닝/data/신문기사/매일경제/(웅진코웨이)%C4%DA%BF%FE%C0%CC/' + str(i)
    for doc in docLabels:
        if doc[:4] == str(i):
            with open(my_path+ '/' + doc, 'r', encoding = 'utf-8') as f2:
                document = f2.read()
                document = re.sub('[a-zA-Z]', '', document)
                document = re.sub('[【】◆•◇ⓒ○→*■ㆍ\t\n\{\}\[\]\/?.,;:|\)*~`!^\-_+<>@\#$%&\\\=\(\'\"]',
                                  '', document)
                data.append(document)
                f2.close() 

In [3]:
#kosac 불러오기
lexi_pos = []
lexi_neg = []
    
f_pos = open('C:/Users/rjsfu/Dropbox/과제/텍스트마이닝/KOSAC/subjectivity-polarity.csv','r',encoding = 'utf-8')    #현재 경로의 exam.csv를 연다.
csvReader_pos = csv.reader(f_pos)    #reader로 파일을 읽는다.

f_neg = open('C:/Users/rjsfu/Dropbox/과제/텍스트마이닝/KOSAC/subjectivity-polarity.csv','r',encoding = 'utf-8')    #현재 경로의 exam.csv를 연다.
csvReader_neg = csv.reader(f_neg)    #reader로 파일을 읽는다.
        
for i in csvReader_pos:            #한 행씩 돌면서 i[2]값 (3번째 컬럼)을 가져와서 리스트에 저장한다.
    lexi_pos.append(i[0]) #홀수는 ngram
    lexi_pos.append(i[5]) #짝수는 pos 확률
f_pos.close

for i in csvReader_neg:            #한 행씩 돌면서 i[2]값 (3번째 컬럼)을 가져와서 리스트에 저장한다.
    lexi_neg.append(i[0]) #홀수는 ngram
    lexi_neg.append(i[3]) #짝수는 neg 확률
f_neg.close
    

lexi_pos = lexi_pos[2:]
lexi_neg = lexi_neg[2:]

In [4]:
#긍정 확률이 0.5보다 큰것만 가져오기
ngram = [lexi_pos[i] for i in range(0,len(lexi_pos)) if i % 2 == 0]
pos_prob = [float(lexi_pos[i]) for i in range(0, len(lexi_pos)) if i % 2 ==1]
pos_dict = {key: value for key,value in zip(ngram, pos_prob) if 0.5 < value}
pos_word = []
for key in pos_dict.keys():
    pos_word.append(key)
    
#부정 확률이 0.5보다 큰것만 가져오기
ngram = [lexi_neg[i] for i in range(0,len(lexi_neg)) if i % 2 == 0]
neg_prob = [float(lexi_neg[i]) for i in range(0, len(lexi_neg)) if i % 2 ==1]
neg_dict = {key: value for key,value in zip(ngram, neg_prob) if 0.5 < value }
neg_word = []
for key in neg_dict.keys():
    neg_word.append(key)

In [10]:
#문서를 명사 형태로 분리
tagger = Twitter()
tokens_noun = []
process_time= time.time()
for text in data:
    tokens_noun.append(tagger.nouns(text))
process_time = time.time() - process_time
print('%.3f secs' % (process_time))

#단어 형태로 분리한 결과를 하나의 리스트 안에 집어넣기
article_noun = [] #-> 형태소로 분리하기 전의 데이터 셋

for i in tokens_noun:
    temp = ''
    for j in i:
        temp += j
        temp += ' '
    article_noun.append(temp)

C:\Users\rjsfu\Anaconda3\lib\site-packages\konlpy\tag\_okt.py:16: UserWarning: "Twitter" has changed to "Okt" since KoNLPy v0.4.5.
  warn('"Twitter" has changed to "Okt" since KoNLPy v0.4.5.')


24.700 secs


In [67]:
#형태소로 분리
pos_tagger = Komoran()
tokens = []
process_time = time.time()
for text in article_noun:
    tokens.append(pos_tagger.pos(text))
print('%.3f secs' % (process_time))

#'단어/형태로' 형태로 맞추기
count = 0
for i in tokens:
    tokens_list = []

    for j in i:
        inner = j[0] + '/' + j[1]
        tokens_list.append(inner)
    
    #긍정 단어 숫자
    p = 0
    n = 0

    for k in tokens_list:
        if k in pos_word:
            p += 1
        if k in neg_word:
            n += 1

    print('article {}'.format(count), '# of positive {}'.format(p), '# of negative {}'.format(n), 'sentiment_score {}'.format((p-n)/(p+n)))
    count += 1

1555921503.810 secs
article 0 # of positive 43 # of negative 14 sentiment_score 0.5087719298245614
article 1 # of positive 116 # of negative 20 sentiment_score 0.7058823529411765
article 2 # of positive 58 # of negative 19 sentiment_score 0.5064935064935064
article 3 # of positive 188 # of negative 34 sentiment_score 0.6936936936936937
article 4 # of positive 78 # of negative 11 sentiment_score 0.7528089887640449
article 5 # of positive 17 # of negative 1 sentiment_score 0.8888888888888888
article 6 # of positive 4 # of negative 2 sentiment_score 0.3333333333333333
article 7 # of positive 112 # of negative 30 sentiment_score 0.5774647887323944
article 8 # of positive 65 # of negative 24 sentiment_score 0.4606741573033708
article 9 # of positive 39 # of negative 4 sentiment_score 0.813953488372093
article 10 # of positive 15 # of negative 7 sentiment_score 0.36363636363636365
article 11 # of positive 22 # of negative 6 sentiment_score 0.5714285714285714
article 12 # of positive 12 # of 

In [5]:
data[28]

'     국가기술표준원이 중금속 검출 논란이 일고 있는 코웨이 얼음정수기 조사에 나선다  한 매체에 따르면 4일 산업통상자원부 국가기술표준원의 정수기 관리 실무부서가 코웨이 얼음정수기에 대한 조사를 결정했다 국표원 관계자는 “얼음정수기의 유해성 논란을 보고받고 신속하게 현장조사에 나서기로 결정했다”고 밝혔다 국표원이 조사에 나선 이유는 제품의 결함가능성 때문이다 정수기 내부에는 얼음을 만드는 부품인 ‘에바’가 있는데 에바의 금속도금이 떨어져 나가면서 물에 니켈이 들어갔을 가능성이 제기됐다 코웨이는 이날 “2014년 4월부터 2015년 12월까지 설치된 얼음정수기 일부 제품에서 니켈 성분이 검출됐고 문제의 제품에 대해서는 97 이상 수리 또는 교환 조치를 취했다”며 “고객분들께 거듭 깊은 사죄의 말씀을 드린다”고 사과했다  코웨이는 세계보건기구 자료를 인용해 “니켈은 내장 흡수가 적고 흡수되지 않은 니켈은 섬유질과 함께 대변으로 배출된다”며 “흡수된 니켈은 소변이나 땀 등 신체 분비물로 배출되는 것으로 보고됐으며 식품이나 음용수로 섭취했을 경우 인체에 축적되지 않는 것으로 알려졌다”고 밝혔다 디지털뉴스국 홍두희 인턴기자  매일경제   무단전재 및 재배포 금지 이미지 키우는 스크립트       '

In [7]:
#KNU불러오기
from os import listdir
from os.path import isfile, join

my_path = 'C:/Users/rjsfu/Dropbox/과제/텍스트마이닝/감성사전'
dic_label = [f for f in listdir(my_path) if f.endswith('.txt')]

dic = []
for i in dic_label:
    with open(my_path + '/' + i, 'r', encoding = 'utf-8') as f2:
        file = f2.read()
        dic.append(file)
        f2.close()
        
negative = dic[0]
positive = dic[2]
neutral = dic[1]

positive = positive.split('\n')
negative = negative.split('\n')
neutral = neutral.split('\n')

negative[0] = negative[0][1:5]
positive[0] = positive[0][1:5]
neutral[0] = neutral[0][1:5]

In [8]:
positive

['가격이 ',
 '가까이 사귀어',
 '가까이하다',
 '가다듬어',
 '가다듬어 수습하는',
 '가다듬어 수습하다',
 '가다듬어 정하다',
 '가당히',
 '가뜬하게',
 '가라앉혀 바로잡다',
 '가르치며',
 '가르침',
 '가벼운 마음으로',
 '가볍게 웃다',
 '가볍고',
 '가볍고 보드랍게',
 '가분가분',
 '가분히',
 '가뿐가뿐',
 '가뿐가뿐하다',
 '가뿐가뿐히',
 '가뿐하게',
 '가뿐하다',
 '가뿐한',
 '가뿐한 느낌',
 '가뿐한 느낌이',
 '가뿐히',
 '가장 중요한',
 '가지런하게 하는',
 '가지런하게 하다',
 '가지런하다',
 '가치',
 '가치가',
 '가치있는',
 '각별나다',
 '각별하다',
 '간직하다',
 '감각적이다',
 '감당하다',
 '감동시킬',
 '감동시킬 만한',
 '감모',
 '감싸고',
 '감정이 북받쳐',
 '강렬하게',
 '강세',
 '강인하다',
 '강하게',
 '강하게 하다',
 '강하고',
 '강하다',
 '강화하다',
 '갖추다',
 '개선하고',
 '개성',
 '개성있는',
 '개성있다',
 '갠소',
 '갸륵한',
 '갸륵한 데가',
 '갸름하다',
 '거리낌 없이',
 '거짓 없다',
 '거짓없다',
 '거침없이',
 '거침이 없는',
 '걱정이 없는',
 '건강에 긍정적인',
 '건둥건둥',
 '건둥건둥하다',
 '검소하다',
 '겉면이 매끄럽고',
 '겉모양의 아름다움과',
 '격려되다',
 '격려하다',
 '격식에 맞아',
 '견고하다',
 '견디어 내다',
 '견줄 수 없을 정도로',
 '결점이 없도록',
 '겸손하게',
 '겸손하게 이르는',
 '겸손하고',
 '경건한',
 '경험이 많고',
 '고상하고',
 '고상한 남자',
 '고상한 이야기',
 '고요하게',
 '고요하게 함',
 '고요하고',
 '고요함',
 '고요해짐',
 '고쳐',
 '고쳐 세우다',
 '고쳐 지음',
 '고쳐 짓다',
 '고치다',
 '곧게',
 '곧게 되다',
 '곧고',
 '곧고 

In [12]:
tokens_noun

[['웅진',
  '코웨이',
  '대표',
  '홍준기',
  '싱글',
  '족',
  '위',
  '초소',
  '역삼투압',
  '정수기',
  '모델',
  '를',
  '출시',
  '이',
  '제품',
  '기존',
  '정수기',
  '절반',
  '크기',
  '가격',
  '렌탈',
  '기준',
  '등록',
  '비',
  '월',
  '렌탈료',
  '시불',
  '디자인',
  '감성',
  '반영',
  '전면',
  '와인',
  '빛',
  '매일경제',
  '무단',
  '재',
  '및',
  '재',
  '배포',
  '금지',
  '이미지',
  '스크립트'],
 ['호텔',
  '신라',
  '인천',
  '국제공항',
  '면세점',
  '계약',
  '웅진',
  '코웨이',
  '자드',
  '애셋',
  '분',
  '추가',
  '취득',
  '해',
  '로',
  '동부',
  '건설',
  '동부',
  '복합',
  '물류',
  '계열',
  '회사',
  '추가',
  '프리',
  '텔',
  '블루',
  '코드',
  '등',
  '주식회사',
  '개',
  '계열',
  '회사',
  '추가',
  '보령제약',
  '바이오나',
  '계',
  '열사',
  '추가',
  '아남전자',
  '남',
  '귀현',
  '최성렬',
  '공동',
  '대표',
  '체제',
  '변경',
  '배명',
  '금속',
  '관계',
  '사',
  '청도',
  '배명',
  '유한',
  '공사',
  '원',
  '규모',
  '채무',
  '보증',
  '결정',
  '씨앤',
  '중공업',
  '양',
  '모씨',
  '부동산',
  '근저당권',
  '설정',
  '등기',
  '말소',
  '관',
  '소송',
  '제기',
  '한진',
  '중공업',
  '원',
  '규모',
  '시설',
  '증설',
  '투자',
  '결정',
  '대한항공',
  '인하',
  '학원